In [1]:
from ipyleaflet import Map, projections, LayersControl, LayerGroup, GeoJSON
from ipyleaflet import ScaleControl, FullScreenControl, SplitMapControl

from localtileserver import examples
from localtileserver import TileClient, get_leaflet_tile_layer

import pystac
import json

In [3]:

with open("/home/jovyan/water-bodies/out.json") as f: 
    results = json.load(f)

In [14]:
for item in results["stac_catalog"]["listing"]:
    
    if item['basename'] == "catalog.json":
        catalog = pystac.read_file(item["path"])
        break
        
catalog.describe()

* <Catalog id=catalog>
  * <Item id=S2B_10TFK_20210713_0_L2A>
  * <Item id=S2A_10TFK_20210718_0_L2A>


In [15]:
m = None

index = 0
groups = []
for item in catalog.get_all_items():

    client = TileClient(item.get_assets()["data"].get_absolute_href())
    t = get_leaflet_tile_layer(client)

    if not m:
        m = Map(center=client.center(), zoom=client.default_zoom)

        control = LayersControl(position='topright')
        m.add_control(control)
        
        groups.append(LayerGroup(name="Area of interest", layers=()))
        m.add_layer(groups[0])
        groups[0].add_layer(GeoJSON(data=item.geometry))
        index = index + 1

    groups.append(LayerGroup(name=item.id, layers=()))
    m.add_layer(groups[index])
    groups[index].add_layer(t)

    index = index + 1 
    
m

Map(center=[40.15302581081341, -121.06648900517655], controls=(ZoomControl(options=['position', 'zoom_in_text'…